In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%capture
!pip install bertopic
!pip install -U kaleido

In [ ]:
import re
import pandas as pd
from datetime import datetime


In [ ]:
ny_post = pd.read_csv('/content/gdrive/MyDrive/Cendie/ny_post(1).csv')
ny_times = pd.read_csv('/content/gdrive/MyDrive/Cendie/ny_times(1).csv')


In [ ]:
df_total = ny_times.append(ny_post).reset_index(drop=True)
df_total = df_total[['Titulo', 'Url', 'Fecha', 'Texto']]
df_total = df_total.dropna(subset = ["Titulo"])
df_total = df_total.reset_index(drop=True)

In [ ]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1629 entries, 0 to 1628
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Titulo  1629 non-null   object
 1   Url     1629 non-null   object
 2   Fecha   1629 non-null   object
 3   Texto   1629 non-null   object
dtypes: object(4)
memory usage: 51.0+ KB


In [ ]:
df_total.Texto = df_total.apply(lambda row: re.sub(r"http\S+", "", row.Texto).lower(), 1)
df_total.Texto= df_total.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.Texto.split())), 1)
df_total.Texto = df_total.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.Texto).split()), 1)
df_total_Texto = df_total.Texto.to_list()

In [ ]:
df_total_texto= []
for i in range(len(df_total_Texto)):
  a = df_total_Texto[i].replace("sign up for our special edition newsletter to get a daily update on the coronavirus pandemic ", "") 
  df_total_texto.append(a)

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
texto_dftotal = pd.Series(df_total_texto)

In [ ]:
total_texto= []

for i in range(len(texto_dftotal)):
  #text_tokens = [word for word in nytimes_text[i] if not word in stopwords.words()]
  #nytimes_texto.append(text_tokens)
  pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
  text = pattern.sub('', texto_dftotal[i])
  total_texto.append(text)


In [ ]:
texto_dftotal = pd.Series(total_texto)

In [ ]:
timestamps= pd.to_datetime(df_total['Fecha'], errors='coerce', utc=True)
timestamps = timestamps.dt.date
len(timestamps)

1629

In [ ]:
df_combinados = pd.concat([texto_dftotal, timestamps], axis = 1)
df_combinados = df_combinados.rename(columns={0: "Texto", 1: "Fecha"})
df_combinados

,Texto,Fecha
0,beijing authorities china reported third death...,2020-01-21
1,briefing ended read developments live coronavi...,2020-03-09
2,united states confirmed third case novel coron...,2020-01-26
3,beijing took thousands infections scores death...,2020-02-10
4,wuhan china top chinese health official warned...,2020-07-03
...,...,...
1624,white house chief staff mark meadows sunday de...,2020-10-25
1625,retired usually take seasonal retail work extr...,2020-10-25
1626,white house chief staff mark meadows said sund...,2020-10-25
1627,nfl finally punished tennessee titans team cov...,2020-10-25


In [ ]:
import datetime
startdate = datetime.datetime.strptime('2020-03-01', "%Y-%m-%d").date()
enddate = datetime.datetime.strptime('2020-10-31', "%Y-%m-%d").date()
mask = (df_combinados['Fecha'] >= startdate) & (df_combinados['Fecha'] <= enddate)
df_combinados1 = df_combinados.loc[mask]
df_combinados1 = df_combinados1.reset_index()
df_combinados1

,index,Texto,Fecha
0,1,briefing ended read developments live coronavi...,2020-03-09
1,4,wuhan china top chinese health official warned...,2020-07-03
2,5,article part developing coronavirus coverage m...,2020-03-09
3,6,briefing ended read latest developments corona...,2020-03-12
4,7,bangkok myanmar loudspeakers broadcast advice ...,2020-09-25
...,...,...,...
1521,1624,white house chief staff mark meadows sunday de...,2020-10-25
1522,1625,retired usually take seasonal retail work extr...,2020-10-25
1523,1626,white house chief staff mark meadows said sund...,2020-10-25
1524,1627,nfl finally punished tennessee titans team cov...,2020-10-25


In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(language="english", calculate_probabilities = True)#, min_topic_size=10, nr_topics=25 , verbose=True)
topics, probs = topic_model.fit_transform(df_combinados1['Texto'])

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
freq = topic_model.get_topic_info(); freq.head(30)

,Topic,Count,Name
0,-1,578,-1_mr_trump_coronavirus_would
1,0,98,0_players_season_game_league
2,1,83,1_city_york_transit_blasio
3,2,71,2_music_theater_museum_show
4,3,60,3_students_school_schools_campus
5,4,52,4_restaurant_restaurants_business_stores
6,5,50,5_family_father_died_mother
7,6,47,6_workers_employees_unemployment_job
8,7,42,7_spain_italy_coronavirus_europe
9,8,42,8_hospital_patients_care_virus


In [ ]:
for i in range(1, 29):    
    topic_nr = freq.iloc[i]["Topic"]
    print(f'Tópico {i-1}:')
    print(topic_model.get_topic(topic_nr)) 

Tópico 0:
[('players', 0.036794749957634446), ('season', 0.031911352541106254), ('game', 0.03166239389161809), ('league', 0.025907640518374967), ('baseball', 0.01591781566820562), ('player', 0.012354712913106981), ('nfl', 0.010983760984205497), ('quarterback', 0.010420600747901763), ('mets', 0.01025151025192513), ('yankees', 0.010194827005518338)]
Tópico 1:
[('city', 0.032761970919685676), ('york', 0.026785968723623977), ('transit', 0.014029416033322827), ('blasio', 0.013134319965311364), ('coronavirus', 0.010982253859765326), ('mayor', 0.01051063996786402), ('yorkers', 0.01004243845549884), ('workers', 0.009538595400568459), ('officials', 0.008328214843355207), ('subway', 0.008257584022997842)]
Tópico 2:
[('music', 0.014179511820084471), ('theater', 0.011649534967554926), ('museum', 0.011231675596416718), ('show', 0.010491385220219975), ('concert', 0.00879576610493726), ('theaters', 0.008512031714455413), ('film', 0.007652379629262222), ('shows', 0.007451687243658495), ('performances'

IndexError: ignored

In [ ]:
fig = topic_model.visualize_hierarchy()
fig.update_layout(title_text = 'Dendograma - Ambos diarios')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/Dendograma.jpg")

In [ ]:
fig

In [ ]:
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/Dendograma.html")

In [ ]:
fig = topic_model.visualize_barchart(n_words=10)
fig.update_layout(title_text = 'Tópicos- Ambos diarios')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/score-topics(1).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/score-topics(1).html")
fig

In [ ]:
fig = topic_model.visualize_barchart([6,7,8,9, 10, 11],n_words=10)
fig.update_layout(title_text = '')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/score-topics(2).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/score-topics(2).html")
fig

In [ ]:
fig = topic_model.visualize_barchart([12,13,14,15,16,17], n_words=10)
fig.update_layout(title_text = '')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/score-topics(3).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/score-topics(3).html")
fig

In [ ]:
fig = topic_model.visualize_barchart([18, 19, 20, 21, 22, 23], n_words=10)
fig.update_layout(title_text = '')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/score-topics(4).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/score-topics(4).html")
fig

In [ ]:
fig = topic_model.visualize_barchart([24,25], n_words=10)
fig.update_layout(title_text = '')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/score-topics(5).jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/score-topics(5).html")
fig

In [ ]:
fig

In [ ]:
fig = topic_model.visualize_heatmap()
fig.update_layout(title_text = 'Matriz de correlación')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/Matriz de correlación.jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/Matriz de correlación.html")
fig

In [ ]:
fig = topic_model.visualize_term_rank()
fig.update_layout(title_text = '')

In [ ]:
#timestamps = timestamps.reset_index(drop=True)

In [ ]:
topics_over_time = topic_model.topics_over_time(docs=df_combinados1['Texto'], 
                                                topics=topics, 
                                                timestamps=df_combinados1['Fecha'], 
                                                global_tuning=True,
                                                evolution_tuning=True)
                                                #nr_bins=20)

In [ ]:
fig = topic_model.visualize_topics_over_time(topics_over_time)
fig.update_layout(title_text = 'Tópicos a través del tiempo')
fig.write_image("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/Topics-over time.jpg")
fig.write_html("/content/gdrive/MyDrive/Cendie/Imagenes_ambos/Topivos- over time.html")

In [ ]:
fig

In [ ]:
fig = topic_model.visualize_topics(); fig

In [ ]:
probabilidades = []
for i in range(0, 1629):
  proba = pd.Series(probs[i].argmax())
  proba = proba[0]
  probabilidades.append(proba)

print(probabilidades)
  

In [ ]:
probabilidades = pd.Series(probabilidades, name = 'Topico más probable')


In [ ]:
ny_contopicos = pd.concat([df_total, probabilidades], axis = 1)
ny_contopicos =  ny_contopicos[[ 'Titulo', 'Url', 'Fecha' ,'Texto' ,'Topico más probable']]
ny_contopicos

In [ ]:
ny_contopicos.info()